In [ ]:
#----  Run doublet removal using scrublet accros samples 

# Scrublet github:  https://github.com/AllonKleinLab/scrublet
# Scrublet preprint: https://www.biorxiv.org/content/10.1101/357368v1


In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc

%matplotlib inline
import scrublet as scr
import scipy.io
    
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')  

import os
import re
import glob

sc.settings.set_figure_params(dpi=80)
sc.logging.print_versions()


/home/nehard/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


scanpy==1.4.3 anndata==0.6.22.post1 umap==0.3.9 numpy==1.14.3 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.19.1 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1 


In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

In [3]:
#-- read 10X output

adata = sc.read_10x_mtx('/mypath/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)
adata

... writing an h5ad cache file to speedup reading next time


View of AnnData object with n_obs × n_vars = 525082 × 33538 
    var: 'gene_ids', 'feature_types'

In [4]:
#-- save rawdata h5ad object 
adata.write('./mypath/ProjectName.h5ad')

Trying to set attribute `.var` of view, making a copy.
... storing 'feature_types' as categorical


In [7]:
adata_raw =sc.read('./mypath/H5AD/ProjectName.h5ad')
adata_raw

AnnData object with n_obs × n_vars = 525082 × 33538 
    var: 'gene_ids', 'feature_types'

In [8]:
adata_raw.obs.head(5)

""
index
AAACCTGAGACTTTCG-1
AAACCTGAGGACCACA-1
AAACCTGCACACGCTG-1
AAACCTGCACCACCAG-1
AAACCTGGTAGAGCTG-1


In [9]:
#--- save meta before scruber (BS)
adata_raw.obs.to_csv('./mypath/Meta/ProjectName.csv')

In [10]:
#--- Meta after R (from samples to cells)

Meta = pd.read_csv('./Meta/MetaII_LifeSpan_BS_092819.csv', index_col=0)
Meta.tail(4)

,Samples,Groups,Batches,IDs,Sample_ids,Age,Gender,Names,X10X_Chemistry
TTTGTTGTCAGACCTA-66,66,HC,B21,JB19041,HPIR-051-9,13y,F,HC_16,v3
TTTGTTGTCGCTTGAA-66,66,HC,B21,JB19041,HPIR-051-9,13y,F,HC_16,v3
TTTGTTGTCGGTTGTA-66,66,HC,B21,JB19041,HPIR-051-9,13y,F,HC_16,v3
TTTGTTGTCTTCGTAT-66,66,HC,B21,JB19041,HPIR-051-9,13y,F,HC_16,v3


In [11]:
#-- Add MetaData to H5AD object  

adata_raw.obs['Gender']= Meta['Gender']
adata_raw.obs['Samples']= Meta['Samples']
adata_raw.obs['Names']= Meta['Names']


In [12]:
adata_raw.write('/mypath/H5AD/ProjectName.h5ad')

... storing 'Gender' as categorical
... storing 'Names' as categorical
... storing 'Age' as categorical
... storing 'Batches' as categorical
... storing 'Groups' as categorical
... storing 'Sample_ids' as categorical
... storing 'Chemistry' as categorical
... storing 'IDs' as categorical


In [13]:
adata_raw =sc.read('/mypath/H5AD/ProjectName.h5ad')
adata_raw

AnnData object with n_obs × n_vars = 525082 × 33538 
    obs: 'Gender', 'Samples', 'Names', 'Age', 'Batches', 'Groups', 'Sample_ids', 'Chemistry', 'IDs'
    var: 'gene_ids', 'feature_types'

In [27]:
names = np.unique(list(adata_raw.obs['Names']))

In [21]:
#-- create File directory 
!mkdir Files

In [ ]:
#--- run Scrublet across the samples 

for sample in names:
    
    cells_of_interest = adata_raw.obs.loc[adata_raw.obs["Names"] == sample, :].index
    
    adata_i = adata_raw[cells_of_interest, :]
    
    scrub = scr.Scrublet(adata_i.X, expected_doublet_rate=0.06)
    
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)
    scrub.plot_histogram();
    
    print('Running UMAP...')
        
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
        
    print('Done.')
        
    scrub.plot_embedding('UMAP', order_points=True);
    
    #doublet_score= pd.DataFrame({'doublet_scores' :list(doublet_scores),'predicted_doublets': list(predicted_doublets)})
        
    adata_i.obs['doublet_score']=scrub.doublet_scores_obs_
    adata_i.uns['sim_doublet_score']=scrub.doublet_scores_sim_
        
    adata_i.obs[['doublet_score']].to_csv('./Files/'+ sample + "_Scrub.csv")
    
    continue

In [31]:
#--- concatenate different csv files (from individual samples)
#--- crate combined .csv file 

path = r'./Files' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


In [32]:
#-- save the combined csv file 

frame.to_csv("mypath/Files/comb_csv.csv", index=False) 

In [33]:
#Scru = pd.read_csv(".mypathFiles/comb_csv.csv",header=0)

In [35]:
Scru = Scru.set_index('index')
Scru.head(5)

,doublet_score
index,
AAACCTGAGAGCAATT-33,0.015378
AAACCTGAGCGTGAGT-33,0.104762
AAACCTGAGTCATGCT-33,0.008269
AAACCTGCAATCCAAC-33,0.031796
AAACCTGCACGGTAGA-33,0.057644


In [36]:
#-- add doublet scores to the h5ad object 

adata_raw.obs['doublet_score']=Scru.doublet_score

In [39]:
#--- Define threshold to be applyed + define doublet vs. singlet
threshold = 0.18
adata_raw.obs['predicted_doublets_1'] = pd.Categorical(['doublet' if x > threshold else 'singlet' for x in adata_raw.obs.doublet_score])

In [1]:
#-- number of singlet and doublets 

adata_raw.obs.predicted_doublets_1.value_counts()

In [41]:
#-- Extract cells defined as singlets 

cells_of_interest1 = adata_raw.obs.loc[adata_raw.obs["predicted_doublets_1"].isin(["singlet"]), :].index
cells_of_interest1

Index(['AAACCTGAGACTTTCG-1', 'AAACCTGAGGACCACA-1', 'AAACCTGCACACGCTG-1',
       'AAACCTGCACCACCAG-1', 'AAACCTGGTAGAGCTG-1', 'AAACCTGGTTCGGCAC-1',
       'AAACCTGTCGCTTAGA-1', 'AAACGGGAGATATGGT-1', 'AAACGGGAGCGATGAC-1',
       'AAACGGGAGGACAGCT-1',
       ...
       'TTTGGTTCACCTCGTT-66', 'TTTGGTTGTTATGACC-66', 'TTTGTTGAGATCCGAG-66',
       'TTTGTTGCAACGATTC-66', 'TTTGTTGCACGGATCC-66', 'TTTGTTGGTGTCATTG-66',
       'TTTGTTGTCAGACCTA-66', 'TTTGTTGTCGCTTGAA-66', 'TTTGTTGTCGGTTGTA-66',
       'TTTGTTGTCTTCGTAT-66'],
      dtype='object', name='index', length=487145)

In [2]:
#-- subset h5ad using the singlet (and discarding doublets)

adata_Scrub=adata_raw[cells_of_interest1, :]
adata_Scrub


In [43]:
#--save the 'singlet object'
adata_Scrub.write(results_file1)
results_file1=('/mypath/ProjectName.h5ad')
